In [1]:
from utils import *

In [2]:
paths = np.load('src/all_paths.npy')
ped_time = paths[:,0]
ped_x = paths[:,1]
ped_y = paths[:,2]

We will generate some paths from left to rigth. So we need to choose two points between 0 and 1080 (frame height) -- one for each side.

Robot has straight path from point 'start' til point 'end'. Also we need to choose start time.

We have data frames from time 0 to time 120,000 (but in fact we have only every twentieth frame and only every fourth frame in paths interpolation so lets choose a number between 0 and 30,000 and then multiply it by 4).

And we need to decide in how many steps robot will achive destination.
To select reasonable speed let's calculate pedestrians' average speed. It's 35 by x-coordinate and 30 by y-coordinate. Let's assume that required number of steps (NoS) it's maximum between NoS by x with speed 35 and NoS by y with speed 30.

In [3]:
np.random.seed(0)

start = (0, np.random.random_integers(0, FRAME_HEIGHT))
end = (FRAME_WIDTH, np.random.random_integers(0, FRAME_HEIGHT))
start_time = 4 * np.random.random_integers(0, 30000)

Now we have all initial data and can generate the whole robot path.

In [50]:
rob_time = np.arange(start_time, start_time + 4 * number_of_steps, 4)
rob_x = np.empty(0, dtype=np.int64)
rob_y = np.empty(0, dtype=np.int64)

points = [(553, 63), (672, 600), (915, 997), (1618, 358)]

for start, end in zip(points[:-1], points[1:]):
    number_of_steps = int(max(abs(end[0] - start[0]) / 35, abs(end[1] - start[1]) / 30)) + 1
    rob_x = np.append(rob_x, np.linspace(start[0], end[0], number_of_steps).astype(np.int64))
    rob_y = np.append(rob_y, np.linspace(start[1], end[1], number_of_steps).astype(np.int64))

Path is ready. We want know how many pedestrians' paths robot will cross. 

In [33]:
def find_by_point(time, x, y):
    indices = np.arange(NUMBER_OF_PEDESTRIANS)
        
    path = np.array([time, x, y])
        
    for i in indices:
        df = download_pedestrian(i, True)
        df.insert(0, 'index', df.index)
        
        data = np.array(df)
        
        ind = np.where(np.all(data == path, axis=1))[0]
        
        if len(ind) > 0:
            return (i, df.index.values[ind[0]])
        
    return (-1, -1)

In [52]:
answer = 0

for i, time in enumerate(rob_time):
    x = rob_x[i]
    y = rob_y[i]
    r = 10
    indices = np.where(ped_time == time)
    x_in_time = ped_x[indices]
    y_in_time = ped_y[indices]
    
    intersect = np.where((x_in_time > x-r)*(x_in_time < x+r)*(y_in_time > y-r)*(y_in_time < y+r))[0]
    answer += intersect.size
    
    if intersect.size > 0:
        ped_index = find_by_point(time, x_in_time[intersect[0]], y_in_time[intersect[0]])
        print(i, time, x, y, ped_index[0])

0 39380 553 63 3369
2 39388 567 126 3361
3 39392 574 157 3360


In [53]:
answer

3

In [55]:
ped1 = np.array(download_pedestrian(3369))
ped2 = np.array(download_pedestrian(3361))
ped3 = np.array(download_pedestrian(3360))
rob = np.array([rob_x, rob_y]).T

draw_lines(39500, [rob, ped1, ped2, ped3], ['green', 'red', 'blue', 'yellow'], 'img')